# DICOM Standard (2014c) Attributes Retrieval
- This Jupyter notebook is designed to retrieve, process, and analyze DICOM Standard (for example, 2014c) document attribute tables using a GPT Agent. 
- To use this notebook, you will need to enter your OpenAI API Key. Please have your personal API Key ready and provide it when prompted. If you do not have one, you can obtain it from https://platform.openai.com/account/api-keys.
- Standard attribute retrieval is based on these rules (Supplementary Materials):
   - Include attributes marked as "Include"
   - Exclude attributes starting with ">Include" (i.e., "Include" inside a Sequence)
   - Exclude attributes starting with ">" (Sequence sub-tags)
     - Exception: If both the Sequence and its sub-tag ("Sequence" and ">Tag") are Required, include the sub-tag. (Example: In MG IOD, both "View Code Sequence" (Type 1) and "> View Modifier" (Type 2) are included for analysis.)
- Note: Because this notebook uses a GPT Agent to extract attributes from Macro tables, the extracted output may vary across runs. Therefore, all extracted attribute tables must be manually reviewed and validated after retrieval.

In [ ]:
import os
import pandas as pd

In [ ]:
# Set a directory
save_dir = 

In [ ]:
# Enter your OpenAI API Key
open_ai_key = 

## Define Function

### Retrieval

In [ ]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI

# Set OpenAI API key
os.environ["OPENAI_API_KEY"]= open_ai_key

# Fetch HTML text from the given URL
def fetch_html(url: str) -> str:
    response = requests.get(url)
    response.raise_for_status()
    return response.text

# Convert tables in HTML to DataFrames
def extract_tables_from_html(html):
    tables = pd.read_html(html)
    return tables

# Wrap as LangChain tools
tools = [
    Tool(
        name="fetch_html",
        func=fetch_html,
        description="Fetches the HTML content from a given URL"
    ),
    Tool(
        name="extract_tables",
        func= extract_tables_from_html,
        description="Extracts all tables from HTML and returns as DataFrame list"
    )
]


# Initialize GPT agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,  # Important: allow JSON parsing failures
)

# Task instructions
prompt = """
Given the following URL: {url}

Perform the following task step-by-step precisely:

1. **Fetch the complete HTML content** from the provided URL.

2. From the HTML content, **extract all tables**. Find and extract the table titled exactly "{table_title}".

4. Inspect the extracted table for references to additional tables, marked clearly with phrases such as:
   - **"Include Table X-Y"**
   - **"See Table X-Y"**

5. If any such references exist within the table:
   - **Iteratively fetch and extract each additionally referenced table** from their respective sections within the DICOM standard documentation.
   - **Clearly record the source** of each attribute from these additional tables (e.g., "Table 10-23 Exposure Index Macro Attributes") in a separate column named **"Source"**.

6. If referenced tables themselves contain further "Include" or "See Table" references, **continue recursively fetching these tables** until there are no more linked table references left.

7. If a reference leads to a section **without structured tables (only plain text)**:
   - **Parse and extract attribute details directly from the text** ("Attribute Name", "Tag", "Type", "Attribute Description").
   - Identify and use any URLs within this text to guide accurate extraction.
   - Record this textual reference clearly in the "Source" column with an appropriate description, such as "Plain text reference from [URL or section title]".

8. **Combine all attributes extracted** (main table + all recursively referenced tables/text) into a single structured CSV with columns exactly as follows:
   - "Attribute Name"
   - "Tag" (e.g. "(0028,3010)")
   - "Type"
   - "Attribute Description"
   - "Source"

9. Output the result **only** as a structured CSV without explanation, comments, or code blocks.
    - Always wrap every column value in double quotes (" ") in the output **CSV**.
    - Always include the exact table or textual reference in the "Source" column for each attribute.
    - Ensure no referenced tables or textual references are missed.
"""


In [ ]:
# Helper function: takes a URL and table_title, runs the agent, and returns the result
def extract_table_attributes(url: str, table_title: str, verbose: bool = True):
    """
    Helper function to extract table attributes from the DICOM standard document
    
    Parameters:
    -----------
    url : str
        URL of the DICOM standard document
    table_title : str
        Exact title of the table to extract
    verbose : bool, default=True
        Whether to enable verbose output when running the agent
    
    Returns:
    --------
    str
        CSV-formatted string of extracted attributes
    """
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=verbose
    )
    return agent.run(prompt.format(url=url, table_title=table_title))

In [ ]:
from io import StringIO

def csv_text_to_dataframe(text):
    df = pd.read_csv(StringIO(text), dtype=str, on_bad_lines='skip')
    #df['Tag'] = df[['Attribute Name', 'Tag']].astype(str).agg(','.join, axis=1)
    #df['Attribute Name'] = df.index.astype(str)
    #df.reset_index(drop=True, inplace=True)
    print(df.shape[0])
    return df

### Polish

In [ ]:
def remove_sequence_composition(df):
    df_trimmed = df[~df["Attribute Name"].str.contains("Include|>", regex=True)]
    return df_trimmed

In [ ]:
def resolve_c873_multiindex(html_id):
    """
    Resolves the wrong multi-index for DataFrame created from csv_text_to_dataframe(html_id).

    Parameters:
    -----------
    html_id : str
        CSV-formatted string representing the table.

    Returns:
    --------
    pd.DataFrame
        Cleaned DataFrame with columns renamed and first index row dropped.
    """
    df_id = csv_text_to_dataframe(html_id)
    df_id = df_id.reset_index()
    df_id = df_id.rename(columns={
        "level_0": "Attribute Name",
        "level_1": "Tag",
        "level_2": "Type",
        "level_3": "Attribute Description",
        "```csv": "Source"
    })
    df_id = df_id.drop(index=0)
    return df_id

## Retrieval 2014c Standard

### CT Image IOD
1. [Table C.8-3. CT Image Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.2.html#table_C.8-3)

In [ ]:
html_c83 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.2.html#table_C.8-3", 
    table_title="Table C.8-3. CT Image Module Attributes")
df_c83 = csv_text_to_dataframe(html_c83)

#### Polish

In [ ]:
# Remove if `Attribute Name` includes 'Include' or '>'
df_c83 = remove_sequence_composition(df_c83)

In [ ]:
# SAVE
df_ct_2014c = df_c83.copy()
#df_ct_2014c.to_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_CT_260115.xlsx'), index=False)

### MR Image IOD
2. [Table C.8-4. MR Image Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.3.html#table_C.8-4)

In [ ]:
html_c84 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.3.html#table_C.8-4", 
    table_title="Table C.8-4. MR Image Module Attributes")
df_c84 = csv_text_to_dataframe(html_c84)

In [ ]:
# SAVE
df_mr_2014c = df_c84.copy()
#df_mr_2014c.to_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_MR_260115.xlsx'), index=False)

### Digital Mammography X-Ray Image IOD
1. [Table C.8-68. DX Series Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.html#sect_C.8.11.1#table_C.8-68)
2. [Table C.8-73. Mammography Series Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.6.html#table_C.8-73)
3. [Table C.8-69. DX Anatomy Imaged Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.2.html#table_C.8-69)
4. [Table C.8-70. DX Image Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.3.html#table_C.8-70)
5. [Table C.8-71. DX Detector Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.4.html#table_C.8-71)
6. [Table C.8-74. Mammography Image Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.7.html#table_C.8-74)

In [ ]:
html_c868 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.html#sect_C.8.11.1#table_C.8-68", 
    table_title="Table C.8-68. DX Series Module Attributes")
df_c868 = csv_text_to_dataframe(html_c868)

In [ ]:
html_c873 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.6.html#table_C.8-73", 
    table_title="Table C.8-73. Mammography Series Module Attributes")
df_c873 = csv_text_to_dataframe(html_c873) 

In [ ]:
html_c869 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.2.html#table_C.8-69", 
    table_title="Table C.8-69. DX Anatomy Imaged Module Attributes")
df_c869 = csv_text_to_dataframe(html_c869) 

In [ ]:
html_c870 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.3.html#table_C.8-70", 
    table_title="Table C.8-70. DX Image Module Attributes")
df_c870 = csv_text_to_dataframe(html_c870) 

In [ ]:
html_c871 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.4.html#table_C.8-71", 
    table_title="Table C.8-71. DX Detector Module Attributes")
df_c871 = csv_text_to_dataframe(html_c871)

In [ ]:
html_c874 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.11.7.html#table_C.8-74", 
    table_title="Table C.8-74. Mammography Image Module Attributes")
df_c874 = csv_text_to_dataframe(html_c874)

#### Polish
- df_c868: remove `Table 10-11. SOP Instance Reference Macro Attributes` (condition 2)
- df_c873: keep only `Table C.8-73. Mammography Series Module Attributes` (condition 2)
- df_c869: exclude `>Include` (conditions 1, 2)
- df_c871: clean up
- df_c874: remove `Table 8.8-1. Code Sequence Macro Attributes` (condition 2)

In [ ]:
# Drop rows where Source == Table 10-11. SOP Instance Reference Macro Attributes
df_c868 = df_c868[df_c868['Source'] != 'Table 10-11. SOP Instance Reference Macro Attributes']

In [ ]:
# Reformat
df_c873 = resolve_c873_multiindex(html_c873)

# Keep only rows where Source == Table C.8-73. Mammography Series Module Attributes
df_c873 = df_c873[df_c873['Source'] == 'Table C.8-73. Mammography Series Module Attributes']

In [ ]:
# Remove if `Attribute Name` includes 'Include' or '>'
df_c869 = remove_sequence_composition(df_c869)

In [ ]:
# Reformat
df_c871 = resolve_c873_multiindex(html_c871)

# Remove if `Attribute Name` includes 'Include' or '>'
df_c871 = remove_sequence_composition(df_c871)

In [ ]:
# Drop rows where Source == Table 8.8-1. Code Sequence Macro Attributes
df_c874 = df_c874[df_c874['Source'] != 'Table 8.8-1. Code Sequence Macro Attributes']

# Remove if `Attribute Name` includes 'Include' or '>'
df_c874 = remove_sequence_composition(df_c874)

#### Concatenate
- `df_c868` (DX Series) + `df_c873` (Mammography Series) + `df_c869` (DX Anatomy Imaged) + `df_c870` (DX Image) + `df_c871` (DX Detector) + `df_c874` (Mammography Image)

In [ ]:
df_mg_2014c = pd.concat([df_c868, df_c873, df_c869, df_c870, df_c871, df_c874])
print(df_mg_2014c.shape[0])
df_mg_2014c.reset_index(drop=True, inplace=True)
df_mg_2014c.head()

In [ ]:
# SAVE
#df_mg_2014c.to_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_MG_260115.xlsx'), index=False)

### Computed Radiography Image IOD
1. [Table C.8-1. CR Series Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.html#table_C.8-1)
2. [Table C.8-2. CR Image Module Attributes](https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.html#table_C.8-2)

In [ ]:
html_c81 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.html#table_C.8-1", 
    table_title="Table C.8-1. CR Series Module Attributes")
df_c81 = csv_text_to_dataframe(html_c81)

In [ ]:
html_c82 = extract_table_attributes(
    url="https://dicom.nema.org/medical/dicom/2014c/output/chtml/part03/sect_C.8.html#table_C.8-2", 
    table_title="Table C.8-2. CR Image Module Attributes")
df_c82 = csv_text_to_dataframe(html_c82)

#### Polish
- df_c82: include [`Table 10-7 General Anatomy Optional Macro Attributes`] without omissions

In [ ]:
df_107 = df_mr_2014c[df_mr_2014c['Source'] == 'Table 10-7 General Anatomy Optional Macro Attributes'].reset_index(drop=True)
df_108 = df_mr_2014c[df_mr_2014c['Source'] == 'Table 10-8 Primary Anatomic Structure Macro Attributes'].reset_index(drop=True)

#### Concatenate
- `df_c81` (CR Series) + `df_c82` (CR Image) + `df_107`+`df_108`

In [ ]:
df_cr_2014c = pd.concat([df_c81, df_c82, df_107, df_108])
print(df_cr_2014c.shape[0])
df_cr_2014c.head()

In [ ]:
# SAVE
#df_cr_2014c.to_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_CR_260115.xlsx'), index=False)

## Preprocess Retrieved 2014c Standard

In [ ]:
df_ct_2014c = pd.read_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_CT_260115.xlsx'))
df_mr_2014c = pd.read_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_MR_260115.xlsx'))
df_mg_2014c = pd.read_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_MG_260115.xlsx'))
df_cr_2014c = pd.read_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_CR_260115.xlsx'))

print(f"CT: {df_ct_2014c.shape[0]} rows with {df_ct_2014c['Tag'].nunique()} tags")
print(f"MR: {df_mr_2014c.shape[0]} rows with {df_mr_2014c['Tag'].nunique()} tags")
print(f"MG: {df_mg_2014c.shape[0]} rows with {df_mg_2014c['Tag'].nunique()} tags")
print(f"CR: {df_cr_2014c.shape[0]} rows with {df_cr_2014c['Tag'].nunique()} tags")

In [ ]:
df_ct_2014c['IOD'] = 'CT Image IOD'
df_mr_2014c['IOD'] = 'MR Image IOD'
df_mg_2014c['IOD'] = 'Digital Mammography X-Ray Image IOD'
df_cr_2014c['IOD'] = 'Computed Radiography Image IOD'

In [ ]:
df_2014c = pd.concat([df_ct_2014c, df_mr_2014c, df_mg_2014c, df_cr_2014c], ignore_index=True)
df_2014c.groupby(['IOD', 'Type'])['Tag'].nunique()

In [ ]:
# SAVE
#df_2014c.to_excel(os.path.join(save_dir, 'C2014MandatoryModalityspecificModules_Full.xlsx'), index=False)

### Duplicated Tags
- MG: 100 rows with 93 tags

In [ ]:
# MG
mg_dup_ls = df_mg_2014c[df_mg_2014c['Tag'].duplicated()]['Tag'].to_list()
df_mg_dups = df_mg_2014c[df_mg_2014c['Tag'].isin(mg_dup_ls)]
df_mg_dups.sort_values(by='Attribute Name')

In [ ]:
# MG - Trim
# When Attribute Name matches in df_mg_2014c, deduplicate by Source priority

# Define priority (lower is higher)
priority_map = [
    ('Table 10-5', 0),
    ('Table 10-6', 1),
    ('Table C.8-74', 2),
    ('Table C.8-73', 3),
    ('Table C.8-70', 4),
    ('Table C.8-69', 5),
    ('Table C.8-68', 6),
    ('Table 10-8', 7),
]

# Function to return priority index for Source strings
def source_priority(source):
    for pattern, prio in priority_map:
        if pattern in str(source):
            return prio
    return 100  # Lowest priority when not in the list above

# Keep only the highest-priority Source row per Attribute Name group
df_mg_2014c['priority'] = df_mg_2014c['Source'].apply(source_priority)
df_mg_2014c = df_mg_2014c.sort_values(['Attribute Name', 'priority'])
df_mg_2014c = df_mg_2014c.drop_duplicates(subset=['Attribute Name'], keep='first')
df_mg_2014c = df_mg_2014c.drop(columns=['priority'])
print(f"MG(trimmed): {df_mg_2014c.shape[0]} rows with {df_mg_2014c['Tag'].nunique()} tags")


In [ ]:
# Remove if `Attribute Name` includes 'Include' or '>'
df_mg_2014c = remove_sequence_composition(df_mg_2014c)

In [ ]:
print(f"MG: {df_mg_2014c.shape[0]} rows with {df_mg_2014c['Tag'].nunique()} tags")

In [ ]:
# MG - Save the Trimmed version
#df_mg_2014c.to_excel(os.path.join(save_dir, 'DicomStandardDocument_2014c_MG_260115_UniqueTags.xlsx'), index=False)

In [ ]:
print(f"CT: {df_ct_2014c.shape[0]} rows with {df_ct_2014c['Tag'].nunique()} tags")
print(f"MR: {df_mr_2014c.shape[0]} rows with {df_mr_2014c['Tag'].nunique()} tags")
print(f"MG: {df_mg_2014c.shape[0]} rows with {df_mg_2014c['Tag'].nunique()} tags")
print(f"CR: {df_cr_2014c.shape[0]} rows with {df_cr_2014c['Tag'].nunique()} tags")

## Concatenage as a whole reference set

In [ ]:
df_2014c_set = pd.concat([df_ct_2014c, df_mr_2014c, df_mg_2014c, df_cr_2014c], ignore_index=True)
df_2014c_set.groupby(['IOD', 'Type'])['Tag'].nunique()

## Exclude Type 1C/2C

In [ ]:
df_2014c_set = df_2014c_set[(df_2014c_set['Type']!='1C')&(df_2014c_set['Type']!='2C')]
df_2014c_set.groupby(['IOD', 'Type'])['Tag'].nunique()

In [ ]:
# SAVE
#df_2014c_set.to_excel(os.path.join(save_dir, 'C2014MandatoryModalityspecificModules_ReferenceSet.xlsx'), index=False)